In [13]:
# importing the libraries
import pandas as pd
import numpy as np
from mlxtend.frequent_patterns import association_rules,apriori
import warnings   
warnings.filterwarnings('ignore')

In [2]:
# reading the data
data=pd.read_excel('Online_Retail.xlsx')

In [3]:
data

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
...,...,...,...,...,...,...,...,...
541904,581587,22613,PACK OF 20 SPACEBOY NAPKINS,12,2011-12-09 12:50:00,0.85,12680.0,France
541905,581587,22899,CHILDREN'S APRON DOLLY GIRL,6,2011-12-09 12:50:00,2.10,12680.0,France
541906,581587,23254,CHILDRENS CUTLERY DOLLY GIRL,4,2011-12-09 12:50:00,4.15,12680.0,France
541907,581587,23255,CHILDRENS CUTLERY CIRCUS PARADE,4,2011-12-09 12:50:00,4.15,12680.0,France


In [15]:
# doing for uk transactions
data_uk = data[data['Country'] == 'United Kingdom']
data_uk

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
...,...,...,...,...,...,...,...,...
541889,581585,22466,FAIRY TALE COTTAGE NIGHT LIGHT,12,2011-12-09 12:31:00,1.95,15804.0,United Kingdom
541890,581586,22061,LARGE CAKE STAND HANGING STRAWBERY,8,2011-12-09 12:49:00,2.95,13113.0,United Kingdom
541891,581586,23275,SET OF 3 HANGING OWLS OLLIE BEAK,24,2011-12-09 12:49:00,1.25,13113.0,United Kingdom
541892,581586,21217,RED RETROSPOT ROUND CAKE TINS,24,2011-12-09 12:49:00,8.95,13113.0,United Kingdom


##### DATA PREPROCESSING

In [16]:
# taking the features that need for the market basket analysis 
data_uk = data_uk[['InvoiceNo','Description','Quantity']]
data_uk

,InvoiceNo,Description,Quantity
0,536365,WHITE HANGING HEART T-LIGHT HOLDER,6
1,536365,WHITE METAL LANTERN,6
2,536365,CREAM CUPID HEARTS COAT HANGER,8
3,536365,KNITTED UNION FLAG HOT WATER BOTTLE,6
4,536365,RED WOOLLY HOTTIE WHITE HEART.,6
...,...,...,...
541889,581585,FAIRY TALE COTTAGE NIGHT LIGHT,12
541890,581586,LARGE CAKE STAND HANGING STRAWBERY,8
541891,581586,SET OF 3 HANGING OWLS OLLIE BEAK,24
541892,581586,RED RETROSPOT ROUND CAKE TINS,24


In [17]:
data_uk.isnull().sum()

InvoiceNo         0
Description    1454
Quantity          0
dtype: int64

In [18]:
# Removing the null values
data_uk=data_uk[~data_uk['Description'].isnull()]
data_uk.isnull().sum()

InvoiceNo      0
Description    0
Quantity       0
dtype: int64

In [19]:
# check whether there is any credits or returns in transaction and removing those transactions
data_uk[data_uk['Quantity']<=0]

,InvoiceNo,Description,Quantity
141,C536379,Discount,-1
154,C536383,SET OF 3 COLOURED FLYING DUCKS,-1
235,C536391,PLASTERS IN TIN CIRCUS PARADE,-12
236,C536391,PACK OF 12 PINK PAISLEY TISSUES,-24
237,C536391,PACK OF 12 BLUE PAISLEY TISSUES,-24
...,...,...,...
540449,C581490,ZINC T-LIGHT HOLDER STARS SMALL,-11
541541,C581499,Manual,-1
541715,C581568,VICTORIAN SEWING BOX LARGE,-5
541716,C581569,HANGING HEART JAR T-LIGHT HOLDER,-1


In [20]:
# removing the return transactions
data_uk=data_uk[data_uk['Quantity']>=0]
data_uk

,InvoiceNo,Description,Quantity
0,536365,WHITE HANGING HEART T-LIGHT HOLDER,6
1,536365,WHITE METAL LANTERN,6
2,536365,CREAM CUPID HEARTS COAT HANGER,8
3,536365,KNITTED UNION FLAG HOT WATER BOTTLE,6
4,536365,RED WOOLLY HOTTIE WHITE HEART.,6
...,...,...,...
541889,581585,FAIRY TALE COTTAGE NIGHT LIGHT,12
541890,581586,LARGE CAKE STAND HANGING STRAWBERY,8
541891,581586,SET OF 3 HANGING OWLS OLLIE BEAK,24
541892,581586,RED RETROSPOT ROUND CAKE TINS,24


In [21]:
# another method for checking credits or bad debt 
data_uk['InvoiceNo'] = data_uk['InvoiceNo'].astype('str')
data_uk[data_uk['InvoiceNo'].str.startswith('A')]

,InvoiceNo,Description,Quantity
299982,A563185,Adjust bad debt,1
299983,A563186,Adjust bad debt,1
299984,A563187,Adjust bad debt,1


In [22]:
# remove the bad debts
data_uk=data_uk[~data_uk['InvoiceNo'].str.startswith('A')]

In [23]:
# cleaned data----> No Returns,No Nullvalues,No debts
data_uk

,InvoiceNo,Description,Quantity
0,536365,WHITE HANGING HEART T-LIGHT HOLDER,6
1,536365,WHITE METAL LANTERN,6
2,536365,CREAM CUPID HEARTS COAT HANGER,8
3,536365,KNITTED UNION FLAG HOT WATER BOTTLE,6
4,536365,RED WOOLLY HOTTIE WHITE HEART.,6
...,...,...,...
541889,581585,FAIRY TALE COTTAGE NIGHT LIGHT,12
541890,581586,LARGE CAKE STAND HANGING STRAWBERY,8
541891,581586,SET OF 3 HANGING OWLS OLLIE BEAK,24
541892,581586,RED RETROSPOT ROUND CAKE TINS,24


In [24]:
data_1=data_uk.copy()

In [25]:
# creating this dataframe to a structured format for market basket analysis
# Method 1 ----> Groupby
data_1 =data_1.groupby(['InvoiceNo','Description'])['Quantity'].sum()
data_1 # clubed the descriptions under their corresponding InvoiceNo

InvoiceNo  Description                        
536365     CREAM CUPID HEARTS COAT HANGER          8
           GLASS STAR FROSTED T-LIGHT HOLDER       6
           KNITTED UNION FLAG HOT WATER BOTTLE     6
           RED WOOLLY HOTTIE WHITE HEART.          6
           SET 7 BABUSHKA NESTING BOXES            2
                                                  ..
581585     ZINC WILLIE WINKIE  CANDLE STICK       24
581586     DOORMAT RED RETROSPOT                  10
           LARGE CAKE STAND  HANGING STRAWBERY     8
           RED RETROSPOT ROUND CAKE TINS          24
           SET OF 3 HANGING OWLS OLLIE BEAK       24
Name: Quantity, Length: 475184, dtype: int64

In [26]:
data_1=data_1.unstack().fillna(0)
data_1# In this we will have the actual non zero values we need only either 1 or 0

Description,4 PURPLE FLOCK DINNER CANDLES,50'S CHRISTMAS GIFT BAG LARGE,DOLLY GIRL BEAKER,I LOVE LONDON MINI BACKPACK,NINE DRAWER OFFICE TIDY,OVAL WALL MIRROR DIAMANTE,RED SPOT GIFT BAG LARGE,SET 2 TEA TOWELS I LOVE LONDON,SPACEBOY BABY GIFT SET,TOADSTOOL BEDSIDE LIGHT,...,returned,taig adjust,test,to push order througha s stock was,website fixed,wrongly coded 20713,wrongly coded 23343,wrongly marked,wrongly marked 23343,wrongly sold (22719) barcode
InvoiceNo,,,,,,,,,,,,,,,,,,,,,
536365,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
536366,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
536367,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
536368,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
536369,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
581582,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
581583,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
581584,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [27]:
# creating a function to convert all the values to 1 or 0
def convert_to_1(value):
    if value >=1:
        return 1
    else:
        return value

data_1= data_1.applymap(convert_to_1)

In [28]:
data_1

Description,4 PURPLE FLOCK DINNER CANDLES,50'S CHRISTMAS GIFT BAG LARGE,DOLLY GIRL BEAKER,I LOVE LONDON MINI BACKPACK,NINE DRAWER OFFICE TIDY,OVAL WALL MIRROR DIAMANTE,RED SPOT GIFT BAG LARGE,SET 2 TEA TOWELS I LOVE LONDON,SPACEBOY BABY GIFT SET,TOADSTOOL BEDSIDE LIGHT,...,returned,taig adjust,test,to push order througha s stock was,website fixed,wrongly coded 20713,wrongly coded 23343,wrongly marked,wrongly marked 23343,wrongly sold (22719) barcode
InvoiceNo,,,,,,,,,,,,,,,,,,,,,
536365,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
536366,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
536367,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
536368,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
536369,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
581582,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
581583,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
581584,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


##### MARKET BASKET ANALYSIS

In [29]:
# Our data is ready 
# STEP 1 find the support and it need to satisfy the support threshold
# for finding support for all combinations of the product use apriori function
data_support_combo = apriori(data_1,min_support=0.03,use_colnames=True,max_len=4)

In [30]:
data_support_combo

,support,itemsets
0,0.047001,(6 RIBBONS RUSTIC CHARM)
1,0.031939,(60 CAKE CASES VINTAGE CHRISTMAS)
2,0.041394,(60 TEATIME FAIRY CAKE CASES)
3,0.030510,(72 SWEETHEART FAIRY CAKE CASES)
4,0.048156,(ALARM CLOCK BAKELIKE GREEN)
...,...,...
135,0.036117,"(JUMBO SHOPPER VINTAGE RED PAISLEY, JUMBO BAG ..."
136,0.038371,"(JUMBO STORAGE BAG SUKI, JUMBO BAG RED RETROSPOT)"
137,0.033368,"(LUNCH BAG BLACK SKULL., LUNCH BAG RED RETROS..."
138,0.030455,"(LUNCH BAG PINK POLKADOT, LUNCH BAG RED RETROS..."


In [31]:
# STEP 2 ----> Association Rules
data_rules = association_rules(data_support_combo,metric='confidence',min_threshold=0.5)
data_rules

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(ALARM CLOCK BAKELIKE GREEN),(ALARM CLOCK BAKELIKE RED ),0.048156,0.051124,0.030949,0.642694,12.571234,0.028487,2.655640
1,(ALARM CLOCK BAKELIKE RED ),(ALARM CLOCK BAKELIKE GREEN),0.051124,0.048156,0.030949,0.605376,12.571234,0.028487,2.412031
2,(PINK REGENCY TEACUP AND SAUCER),(GREEN REGENCY TEACUP AND SAUCER),0.038645,0.051344,0.031719,0.820768,15.985646,0.029735,5.292898
3,(GREEN REGENCY TEACUP AND SAUCER),(PINK REGENCY TEACUP AND SAUCER),0.051344,0.038645,0.031719,0.617773,15.985646,0.029735,2.515140
4,(ROSES REGENCY TEACUP AND SAUCER ),(GREEN REGENCY TEACUP AND SAUCER),0.052608,0.051344,0.038536,0.732497,14.266445,0.035834,3.546343
5,(GREEN REGENCY TEACUP AND SAUCER),(ROSES REGENCY TEACUP AND SAUCER ),0.051344,0.052608,0.038536,0.750535,14.266445,0.035834,3.797698
6,(JUMBO BAG BAROQUE BLACK WHITE),(JUMBO BAG RED RETROSPOT),0.050025,0.106536,0.031334,0.626374,5.879444,0.026005,2.391330
7,(JUMBO BAG PINK POLKADOT),(JUMBO BAG RED RETROSPOT),0.063713,0.106536,0.043153,0.677308,6.357539,0.036365,2.768782
8,(JUMBO SHOPPER VINTAGE RED PAISLEY),(JUMBO BAG RED RETROSPOT),0.062284,0.106536,0.036117,0.579876,5.442999,0.029481,2.126669
9,(JUMBO STORAGE BAG SUKI),(JUMBO BAG RED RETROSPOT),0.062119,0.106536,0.038371,0.617699,5.798021,0.031753,2.337070


In [32]:
# STEP 3 ---- Lift (value must be greater than 1)
data_final_rules=data_rules[data_rules['lift']>1]
data_final_rules

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(ALARM CLOCK BAKELIKE GREEN),(ALARM CLOCK BAKELIKE RED ),0.048156,0.051124,0.030949,0.642694,12.571234,0.028487,2.655640
1,(ALARM CLOCK BAKELIKE RED ),(ALARM CLOCK BAKELIKE GREEN),0.051124,0.048156,0.030949,0.605376,12.571234,0.028487,2.412031
2,(PINK REGENCY TEACUP AND SAUCER),(GREEN REGENCY TEACUP AND SAUCER),0.038645,0.051344,0.031719,0.820768,15.985646,0.029735,5.292898
3,(GREEN REGENCY TEACUP AND SAUCER),(PINK REGENCY TEACUP AND SAUCER),0.051344,0.038645,0.031719,0.617773,15.985646,0.029735,2.515140
4,(ROSES REGENCY TEACUP AND SAUCER ),(GREEN REGENCY TEACUP AND SAUCER),0.052608,0.051344,0.038536,0.732497,14.266445,0.035834,3.546343
5,(GREEN REGENCY TEACUP AND SAUCER),(ROSES REGENCY TEACUP AND SAUCER ),0.051344,0.052608,0.038536,0.750535,14.266445,0.035834,3.797698
6,(JUMBO BAG BAROQUE BLACK WHITE),(JUMBO BAG RED RETROSPOT),0.050025,0.106536,0.031334,0.626374,5.879444,0.026005,2.391330
7,(JUMBO BAG PINK POLKADOT),(JUMBO BAG RED RETROSPOT),0.063713,0.106536,0.043153,0.677308,6.357539,0.036365,2.768782
8,(JUMBO SHOPPER VINTAGE RED PAISLEY),(JUMBO BAG RED RETROSPOT),0.062284,0.106536,0.036117,0.579876,5.442999,0.029481,2.126669
9,(JUMBO STORAGE BAG SUKI),(JUMBO BAG RED RETROSPOT),0.062119,0.106536,0.038371,0.617699,5.798021,0.031753,2.337070
